### 线性回归算法详解


#### 一、核心概念
线性回归是一种通过建立自变量(特征)与因变量(目标值)之间的**线性关系**来进行预测的监督学习算法。其核心假设是:因变量可由自变量的线性组合(加常数项)表示,即  
$$\hat{y} = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n$$  
其中,$\hat{y}$ 是预测值,$x_1,x_2,...,x_n$ 是自变量,$w_0$ 是截距项,$w_1,w_2,...,w_n$ 是特征权重(待求解参数)。


#### 二、核心原理
1. **目标函数**  
   线性回归通过最小化**预测值与真实值的平方误差和(SSE)** 求解最优参数,即:  
   $$\text{Loss} = \sum_{i=1}^m (\hat{y}_i - y_i)^2 = \sum_{i=1}^m (w_0 + w_1x_{i1} + ... + w_nx_{in} - y_i)^2$$  
   该目标函数也称为**均方误差(MSE)** 的总和(除以样本量 $m$ 即为 MSE)。

2. **参数求解**  
   - **解析解(最小二乘法)**:通过对目标函数求导并令导数为 0,可直接求解参数 $w$ 的解析解:  
     $$\hat{w} = (X^TX)^{-1}X^Ty$$  
     其中 $X$ 是包含截距项(全为 1 的列)的特征矩阵,$y$ 是真实值向量。  
   - **数值解法**:当特征维度高或 $X^TX$ 不可逆时(如多重共线性),可通过梯度下降、随机梯度下降(SGD)等迭代方法求解参数。


#### 三、扩展变体
1. **多元线性回归**:自变量数量 ≥ 2(区别于单变量线性回归)。  
2. **多项式回归**:通过对特征进行多项式转换(如 $x \to x^2, x^3$),将非线性关系转化为线性回归问题(本质仍是线性回归,参数为多项式系数)。  
3. **正则化线性回归**:  
   - **岭回归(Ridge Regression)**:在损失函数中加入 $L_2$ 正则项($\lambda\sum w_i^2$),缓解多重共线性,防止过拟合。  
   - **Lasso 回归**:加入 $L_1$ 正则项($\lambda\sum |w_i|$),可实现特征选择(使部分 $w_i=0$)。  
   - **Elastic Net**:结合 $L_1$ 和 $L_2$ 正则项,平衡特征选择与稳定性。  


#### 四、适用场景
- **任务类型**:仅适用于**回归任务**(预测连续值,如房价、销售额、温度等)。  
- **数据要求**:  
  - 特征与目标值存在**线性相关性**(可通过可视化或相关性分析验证)。  
  - 误差项满足**独立性、同方差性、正态分布**(违反时需预处理,如对数变换、去除异常值)。  
- 典型场景:  
  - 房价预测(基于面积、房间数等特征)。  
  - 销售额预测(基于广告投入、客流量等特征)。  
  - 学生成绩预测(基于学习时长、出勤率等特征)。  


#### 五、实践要点
1. **特征预处理**:  
   - **标准化/归一化**:消除量纲影响(如梯度下降时加速收敛)。  
   - **处理多重共线性**:通过方差膨胀因子(VIF)检测,删除高相关特征或使用岭回归。  
   - **缺失值处理**:填充(均值、中位数)或删除缺失样本。  

2. **过拟合与欠拟合**:  
   - 欠拟合:模型过于简单(如线性关系假设不成立),需增加特征复杂度(如多项式转换)。  
   - 过拟合:模型过于复杂(如高次多项式),需简化模型、增加正则化强度或增大样本量。  

3. **模型评估**:需结合任务需求选择评估指标(见下文)。  


#### 六、评估指标怎么选？
线性回归的评估指标用于衡量预测值与真实值的差异,选择需结合**业务场景**和**数据特点**:  

| 指标名称       | 公式(针对测试集)                          | 特点与适用场景                                                                 |
|----------------|-------------------------------------------|------------------------------------------------------------------------------|
| 均方误差(MSE) | $\text{MSE} = \frac{1}{m}\sum (\hat{y}_i - y_i)^2$ | 对异常值敏感(平方放大误差),适用于关注误差大小的场景(如房价预测,大误差影响大)。 |
| 均方根误差(RMSE) | $\text{RMSE} = \sqrt{\text{MSE}}$          | 与目标值同量纲,更易解释(如预测房价时,RMSE=5 表示平均误差 5 万元)。           |
| 平均绝对误差(MAE) | $\text{MAE} = \frac{1}{m}\sum \vert\hat{y}_i - y_i\vert$ | 对异常值稳健,适用于异常值较多的场景(如销售额预测,避免极端值干扰)。           |
| 决定系数($R^2$) | $R^2 = 1 - \frac{\sum (\hat{y}_i - y_i)^2}{\sum (y_i - \bar{y})^2}$ | 表示模型解释的方差比例(范围 [0,1]),越接近 1 说明拟合越好,适用于需要直观判断模型整体性能的场景。 |

- **优先选 $R^2$**:快速判断模型整体拟合效果,不受量纲影响。  
- **关注 RMSE/MAE**:当需要具体误差数值(如“平均预测偏差多少元”)时使用,异常值多则选 MAE。  


#### 七、代码实例(3种)

##### 1. 简洁版:手撕线性回归(最小二乘法)

In [ ]:
import numpy as np

# 生成样本数据(y = 3 + 2x1 + 5x2 添加截距项后的X+ 噪声)
np.random.seed(42)  # 设置随机种子以便复现结果
X = np.random.rand(100, 2)  # 2个特征,100个样本
y = 3 + 2 * X[:, 0] + 5 * X[:, 1] + np.random.normal(0, 0.1, 100)  # 加入噪声
"""
    np.random.normal(0, 0.1, 100) 的作用是生成 100 个
    服从正态分布的随机数,这些数的均值为 0,标准差为 0.1。
"""
print("X:\n", X[:5])
print("\ny:\n", y.shape)

# 给X添加截距项(全为1的列)
X_b = np.hstack([np.ones((X.shape[0], 1)), X])
"""
    1. X.shape[0] 表示矩阵 X 的行数（即样本数量），shape方括号里的数字表示维度。
    2. np.ones((X.shape[0], 1)): 该函数用于创建一个二维 numpy 数组，形状为 (X.shape[0], 1)。
    3. np.hstack(): 该函数用于将多个数组在水平方向（列方向）上拼接起来。这里将全 1 数组和特征矩阵 X 拼接，
"""
print("\nX_b(添加截距项后的X):\n", X_b[:5])

# 最小二乘法求解参数
w = np.linalg.inv(X_b.T @ X_b) @ X_b.T @y

print("\n参数w(截距项+权重):\n", w)  # 应接近 [3, 2, 5]

X:
 [[0.37454012 0.95071431]
 [0.73199394 0.59865848]
 [0.15601864 0.15599452]
 [0.05808361 0.86617615]
 [0.60111501 0.70807258]]

y:
 (100,)

X_b(添加截距项后的X):
 [[1.         0.37454012 0.95071431]
 [1.         0.73199394 0.59865848]
 [1.         0.15601864 0.15599452]
 [1.         0.05808361 0.86617615]
 [1.         0.60111501 0.70807258]]

参数w(截距项+权重):
 [2.97722723 2.03386668 5.0354946 ]


##### 2. 简洁版:sklearn 实现线性回归

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

# 生成数据(同上)
np.random.seed(42)
X = np.random.rand(100, 2)
y = 3 + 2*X[:,0] + 5*X[:,1] + np.random.normal(0, 0.1, 100)
print("X:\n", X[:5],"\n")
print("y:\n", y[:5],"\n")
print("y.shape:\n", y.shape,"\n")

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
model = LinearRegression()
model.fit(X_train, y_train)

# 输出参数与预测
print("截距项:\n", model.intercept_,"\n")  # 接近3
print("权重:\n", model.coef_,"\n")  # 接近[2,5]

print("测试集特征值：\n", X_test[:5],"\n")
print("测试集目标值：\n", y_test[:5],"\n")
print("预测结果：\n",model.predict(X_test)[:5])

X:
 [[0.37454012 0.95071431]
 [0.73199394 0.59865848]
 [0.15601864 0.15599452]
 [0.05808361 0.86617615]
 [0.60111501 0.70807258]] 

y:
 [8.4346493  7.48050567 4.12131713 7.37561281 7.92917036] 

y.shape:
 (100,) 

截距项:
 2.971667149938966 

权重:
 [2.02545136 5.04741599] 

测试集特征值：
 [[0.32078006 0.18651851]
 [0.41038292 0.75555114]
 [0.96244729 0.2517823 ]
 [0.11959425 0.71324479]
 [0.88721274 0.47221493]] 

测试集目标值：
 [4.63754458 7.50473903 6.25243209 6.82078494 7.04711437] 

预测结果：
 [4.56282808 7.6164587  6.19190731 6.81394262 7.15213857]


##### 3. 全面版:sklearn 实现(含预处理、正则化、评估)

In [15]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

# 1. 生成复杂数据(含多重共线性)
np.random.seed(42)
X1 = np.random.rand(1000)  # 生成 1000 个服从 [0, 1) 均匀分布的随机数作为特征 1
X2 = 0.8*X1 + 0.2*np.random.rand(1000)  # 生成与 X1 高度相关的特征 2，引入部分随机噪声以模拟多重共线性
X3 = np.random.rand(1000)
X = np.column_stack([X1, X2, X3])  # 将三个特征按列合并成一个二维数组，作为特征矩阵 X
y = 1.2 + 3.5*X1 - 2.1*X2 + 4.8*X3 + np.random.normal(0, 0.3, 1000)
print("X1:\n", X1[:5],"\n")
print("X2:\n", X2[:5],"\n")
print("X3:\n", X3[:5],"\n")
print("X:\n", X[:5],"\n")
print("y:\n", y[:5],"\n")

X1:
 [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864] 

X2:
 [0.33665868 0.86895163 0.76018432 0.62537176 0.28612714] 

X3:
 [0.26170568 0.2469788  0.90625458 0.2495462  0.27194973] 

X:
 [[0.37454012 0.33665868 0.26170568]
 [0.95071431 0.86895163 0.2469788 ]
 [0.73199394 0.76018432 0.90625458]
 [0.59865848 0.62537176 0.2495462 ]
 [0.15601864 0.28612714 0.27194973]] 

y:
 [3.37807538 4.07330166 6.72068451 2.77005903 2.81414012] 



In [16]:
# 2. 数据预处理(标准化)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("X_scaled:\n",X_scaled[:5],"\n")

X_scaled:
 [[-0.39630103 -0.64737723 -0.82848938]
 [ 1.57695733  1.54819008 -0.8791793 ]
 [ 0.82789256  1.09955374  1.39004747]
 [ 0.37125061  0.54348769 -0.87034231]
 [-1.14468466 -0.85580643 -0.79322939]] 



In [18]:
# 3. 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 4. 训练模型(普通线性回归 vs 岭回归)
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)  # alpha为正则化强度

model_lr.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)

# 5. 模型对比
print("普通线性回归参数:")
print(f"截距:{model_lr.intercept_:.4f}，权重:{model_lr.coef_}")
print("\n岭回归参数(缓解多重共线性):")
print(f"截距:{model_ridge.intercept_:.4f}，权重:{model_ridge.coef_}")

普通线性回归参数:
截距:4.2953，权重:[ 1.10554446 -0.60520105  1.39628486]

岭回归参数(缓解多重共线性):
截距:4.2951，权重:[ 1.07026118 -0.57026211  1.39433284]


In [21]:
# 6. 交叉验证评估
cv_scores_lr = cross_val_score(model_lr, X_scaled, y, cv=5, scoring='r2')  #  scoring='r2' 表示使用决定系数（R²）作为评估指标
cv_scores_ridge = cross_val_score(model_ridge, X_scaled, y, cv=5, scoring='r2')
print(f"\n普通线性回归交叉验证R²:{cv_scores_lr.mean():.4f}(±{cv_scores_lr.std():.4f})")  # 标准差体现得分的波动情况
print(f"岭回归交叉验证R²:{cv_scores_ridge.mean():.4f}(±{cv_scores_ridge.std():.4f})")


普通线性回归交叉验证R²:0.9611(±0.0020)
岭回归交叉验证R²:0.9610(±0.0020)


In [22]:
# 7. 测试集评估
y_pred_lr = model_lr.predict(X_test)
y_pred_ridge = model_ridge.predict(X_test)
print("\n测试集评估:")
print(f"普通线性回归 R²:{r2_score(y_test, y_pred_lr):.4f},MSE:{mean_squared_error(y_test, y_pred_lr):.4f}")
print(f"岭回归 R²:{r2_score(y_test, y_pred_ridge):.4f},MSE:{mean_squared_error(y_test, y_pred_ridge):.4f}")


测试集评估:
普通线性回归 R²:0.9614,MSE:0.0999
岭回归 R²:0.9618,MSE:0.0989
